In [ ]:
from sequence.protocol import Protocol
from sequence.message import Message
from enum import Enum, auto

class HandshakeState(Enum):
    CLOSED = auto()
    SYN_SENT = auto()
    SYN_RECEIVED = auto()
    ESTABLISHED = auto()
    FAILED = auto()

class HandshakeMessageType(Enum):
    SYN = auto()
    SYN_ACK = auto()
    ACK = auto()
    
# Vai ser instanciado no sender e no receiver
class EntangleConnectionHandshakeMessage(Message):
    """
    A mensagem é composta por um message_type (SYN, SYN_ACK, ACK), um session_id (id da sessão/tentativa de entrelaçamento) e um memory_id (id da memória que vai ser usada).
    """
    def __init__(self, msg_type, session_id, protocol_receiver, memory_id=None):
        # isso ajuda no roteamento da mensagem, o node identifica se protocol.name == msg.receiver
        # DETALHE: o protocol_receiver é o protocolo que vai receber a mensagem, e interprear ela
        super().__init__(msg_type, protocol_receiver)
        self.session_id = session_id
        # o memory deve mudar de acordo com o nó que tem o protocolo
        self.memory_id = memory_id

    def __str__(self):
        return f"{self.msg_type.name} | session: {self.session_id} | mem: {self.memory_id}"

class EntangleConectionProtocol(Protocol):
    """
    Protocolo de entrelaçamento entre dois nós.
    """
    def __init__(self, owner, name, remote_node):
        # owner é o nó que tem o protocolo, name é o nome do protocolo
        super().__init__(owner, name)
        # remote_node é o nó que vai ser entrelaçado, owner é o nó que tem o protocolo
        self.remote_node = remote_node
        self.session_id = f'[{self.owner.name}-{self.remote_node.name}]{id(self)}'
    
    def start(self):
        """
        Inicia o protocolo de entrelaçamento, entre A e B. Sendo A o que solicita o entrelaçamento e B o que responde.
        """
        # dependendo, pode ser chamado com um destino diferente, nesse caso, o remote_node vai mudando aqui
        # aloca a memória e faz o handshake
        # envia a mensagem de SYN para o remote_node
        self.send_message(self.remote_node, HandshakeMessageType.SYN)
        
        # muda o estado para SYN_SENT
        self.set_state(HandshakeState.SYN_SENT)
    
    def send_message(self, dest, msg_type):
        """
        Envia a mensagem para o remote_node.
        
        Args:
            dest (str): O nó de destino.
            msg_type (Enum): O tipo da mensagem.
        """
        # marca o protocolo que vai receber a mensagem
        receiver_protocol = "ech2-{owner.name}"
        
        # se o tipo da mensagem for SYN, então tem que esperar o SYN_ACK para enviar a memória que vamos usar
        if msg_type != HandshakeMessageType.SYN:
            memory_id = self.memory.id # tá eviando quando é ack tbm # TODO: verificar se é isso mesmo
        else:
            memory_id = None
            
        # cria a mensagem de handshake
        msg = EntangleConnectionHandshakeMessage(
            msg_type=msg_type,
            session_id=self.session_id,
            protocol_receiver=receiver_protocol,
            memory_id=memory_id
        )
        
        # envia a mensagem para o nó de destino
        self.owner.send_message(dest, msg)
        
        
    def received_message(self, src, msg):
        """
        Recebe a mensagem do remote_node e processa de acordo com o estado do protocolo.
        """
        if msg.msg_type == HandshakeMessageType.SYN:      
            # então sou um B, tem um hub querendo se conectar
            # aloca a memória que vai ser usada para o entrelaçamento
            self.memory = self.owner.allocate_raw_memory()
            
            # se nenhuma memória disponível
            if self.memory is None:
                print("Nenhuma memória disponível")
                return
            
            # responde com um SYN_ACK
            self.send_message(src, HandshakeMessageType.SYN_ACK)
            
            # muda o estado para SYN_RECEIVED
            self.set_state(HandshakeState.SYN_RECEIVED)
            
        elif msg.msg_type == HandshakeMessageType.SYN_ACK:
            # então sou um A, já enviei o SYN e recebi o SYN_ACK
            self.owner.allocate_memory(msg.memory_id)
            
            # se não tiver memória disponível, então encerra o protocolo
            if self.memory is None:
                print("Nenhuma memória disponível")
                return
            
            # recebi qual a memória que o receiver quer usar, então posso enviar a mensagem de ACK com o id da memória que vou usar
            # responde com um ACK
            self.send_message(src, HandshakeMessageType.ACK)
            
            # muda o estado para ESTABLISHED
            self.set_state(HandshakeState.SYN_RECEIVED)
            
        elif msg.msg_type == HandshakeMessageType.ACK:
            # então sou um B, já enviei o SYN_ACK e recebi o ACK
            
            # finaliza o handshake
            self.set_state(HandshakeState.FINISHED)
            
    
    
    def set_state(self, state):
        # se o estado mudar para FAILED então encerra
        # self.close()
        self.state = state

    # TODO: implementar o close, para fechar a conexão e liberar a memória
    
    

In [ ]:
from sequence.kernel.timeline import Timeline
from sequence.components.memory import MemoryArray
from sequence.topology.node import Node
from sequence.resource_management.resource_manager import ResourceManager

# cria uma instância de Timeline
runtime = 1e12  # tempo máximo de simulação em picosegundos
tl = Timeline(runtime)

# Hub tem que ter memória, gerenciar os entrelaçamentos
class Hub(Node):
    """Nó responsável por gerenciar a memória e os entrelaçamentos. Fica entre os sensores e o gateway de saída."""
    def __init__(self, name, timeline):
        super().__init__(name, timeline)
        # Memória de qubits, são esses argumentos
        memory_array_name = f"{self.name}.memory_array"
        # outros detalhes envolvem parametros de configuração da memória, mas por enquanto só isso
        self.memory_array = MemoryArray(memory_array_name, timeline, num_memories=10)
        # adiciona as memórias ao hub
        self.add_component(self.memory_array)
        # indica que a memória é desse hub
        self.memory_array.add_receiver(self)
        # gerenciar os entrelaçamentos (pode ser melhor definir o próprio, mas por enquanto é assim)
        self.resource_manager = ResourceManager(self, memory_array_name)

    def get(self, photon, **kwargs):
        src = kwargs.get("src")
        dst = kwargs.get("dst")

        # se não for o destino, apenas reencaminha o photon para o destino correto
        if dst != self.name:
            print(f"[{self.name}] qubit de {src} redirecionado para {dst}")
            self.send_qubit(dst, photon)
            return

        # tenta alocar memória RAW
        for info in self.resource_manager.memory_manager:
            if info.state == "RAW":
                # isso é opcional pq é meio que de mentirinha, o quantummanager faz o trabalho de verdade
                info.memory.photon = photon
                print(f"[{self.name}] armazenou qubit de {src} em {info.memory.name}")
                return

        # se não encontrou memória, descarta o fóton
        print(f"[{self.name}] MEMÓRIA CHEIA. Qubit de {src} descartado.")
    
    def alocate_raw_memory(self):
        # TODO: verificar se esse é o mesmo caso do get, pra mim, parece diferente. não sei ainda por que
        memory = self.check_raw_memory()
        if memory is None:
            return 
        memory.state = "ALLOCATED"
        print(f"[{self.name}] alocou memória {memory.name} para {self.session_id}")
        
    
    def check_raw_memory(self):
        # verifica se tem memória RAW disponível
        for info in self.resource_manager.memory_manager:
            if info.state == "RAW":
                return info.memory
        
        
hub = Hub("hub", tl)
for m in hub.resource_manager.memory_manager:
    print(type(m))
    print(f"[Hub {hub.name}] memória {m.memory.name} está {m.state}.")
# configuração opcional: desativar barra de progresso
tl.show_progress = False

<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[0] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[1] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[2] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[3] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[4] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[5] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[6] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_array[7] está RAW.
<class 'sequence.resource_management.memory_manager.MemoryInfo'>
[Hub hub] memória hub.memory_ar